# VCF_Reading_Tutorial

Hi guys, this is my quick tutorial how to read vcf data. I am by no means an expert, so you are welcome to look up things on your own and expand/change this notebook. Regardless, for the purposes of this notebook I will be reading just one of the provided files, note that you do not need to extract the file and can instead use the .gz compressed version when working with the chosen library.

## Installing pyvcf

For the purpose of reading vcf files I have chosen pyvcf library and as such, this is what the tutorial is centered around. Other libraries are available so if you find something better suited feel free to use it, but for now let's get the basics of pyvcf out of the way.

Installing pyvcf is unironically one of the hardest parts of using it. For the record, I used Google Colab, so your experience when using some other IDE can be different. Also if you are working on it locally, please use a virtual environment. Regardless, first try a normal install, for example through standard pip command:
* pip install pyvcf

If it does not work(happens often), then use these commands:
* pip install "setuptools<58" --upgrade
* pip uninstall pyvcf
* pip install pyvcf

If you are working on a notebook environment then you can copy the corresponding cells below. If it still does not work, then upgrade pip and install necessary tools, then rerun the previous commands like so:
* pip install --upgrade pip
* !apt-get install -y python-dev libssl-dev libffi-dev libxml2 libxml2-dev libxslt1.1 libxslt1-dev zlib1g-dev
* pip install "setuptools<58" --upgrade
* pip uninstall pyvcf
* pip install pyvcf

In [37]:
!pip install --upgrade pip

In [38]:
!apt-get install -y python-dev libssl-dev libffi-dev libxml2 libxml2-dev libxslt1.1 libxslt1-dev zlib1g-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package python-dev is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source
However the following packages replace it:
  python2-dev python2 python-dev-is-python3

E: Package 'python-dev' has no installation candidate


In [39]:
!pip install "setuptools<58" --upgrade
!pip uninstall pyvcf
!pip install pyvcf

Found existing installation: PyVCF 0.6.8
Uninstalling PyVCF-0.6.8:
  Would remove:
    /usr/local/bin/vcf_filter.py
    /usr/local/bin/vcf_melt
    /usr/local/bin/vcf_sample_filter.py
    /usr/local/lib/python3.10/dist-packages/PyVCF-0.6.8.dist-info/*
    /usr/local/lib/python3.10/dist-packages/vcf/*
Proceed (Y/n)? ERROR: Operation cancelled by user
^C


#### Further troubleshooting

If the issue still presists (probably windows-related) try installing the package through conda

In [ ]:
%%cmd

C:\Users\USER\anaconda3\Scripts\activate base
conda activate pathogen
conda install pyvcf

If this doesn't work try:

In [ ]:
%%cmd

C:\Users\USER\anaconda3\Scripts\activate base
conda activate pathogen

conda config --add channels conda-forge
conda config --set channel_priority strict
conda install pyvcf

Which finally worked for me

## Using pyvcf

So I have never before worked with vcf files, but from what I have learned they have a certain "built-in"(that is not the official term but I do not know how else to call it) fields that are subject to certain constraints and standards and some "custom" fields that the user can define when writing the file. Regardless, let's focus on the built-in fields first. Below is the code for reading first 10 data points and showing information related to them.

In [2]:
import vcf

vcf_file = "data/EE_015.vcf.gz"

with open(vcf_file, 'rb') as vcf_file_binary:
    vcf_reader = vcf.Reader(vcf_file_binary)

    i=0
    for record in vcf_reader:
        i+=1
        if i>10:
            break
        print("CHROM:", record.CHROM)
        print("POS:", record.POS)
        print("ID:", record.ID)
        print("REF:", record.REF)
        print("ALT:", record.ALT)
        print("QUAL:", record.QUAL)
        print("FILTER:", record.FILTER)
        print("INFO:", record.INFO)
        print("FORMAT:", record.FORMAT)

        for sample in record.samples:
            print(f"Sample {sample.sample}: {sample['GT']}")

        print("\n")

CHROM: chr1
POS: 15820
ID: rs2691315
REF: G
ALT: [T]
QUAL: None
FILTER: []
INFO: {'ACMG_class': ['Uncertain%40Significance'], 'ACMG_coding_impact': ['non%40coding'], 'ACMG_rules': ['BP4'], 'ACMG_score': [0.193], 'AMP_matches': ['V'], 'AMP_rules': ['Freq_II%2CType_IV%2CPred_IV'], 'AMP_tier': ['Tier%40IV'], 'AMP_total_samples': [0], 'AS_FilterStatus': 'SITE', 'AS_SB_TABLE': '12%2C23|7%2C2', 'DP': 45, 'ECNT': 1, 'GERMQ': 55, 'Gene': ['WASH7P'], 'MBQ': [29], 'MFRL': [219], 'MMQ': [34], 'MPOS': [30], 'POPAF': [7.3], 'TLOD': [10.9], 'function': ['non-coding%40exon'], 'gnomadGenomesAC': [18059], 'gnomadGenomesAN': [82218], 'gnomadGenomesEthnic_AC_Hom': [233], 'gnomadGenomes_AC_Hom': [18059], 'gnomadGenomes_AF': [0.219648], 'gnomadGenomes_AF_ethnic': [0.0938074], 'CSQ': ['T|downstream_gene_variant|MODIFIER|DDX11L1|ENSG00000223972|Transcript|ENST00000450305.2|transcribed_unprocessed_pseudogene||||||||||rs2691315|2150|1||HGNC|HGNC:37102|YES||||||||||||Ensembl||G|G||||||0.4105|0.4849|0.2939|0.605

Here is the explanation for the above fields(courtesy of ChatGPT):
* CHROM: The chromosome or sequence name where the variant is located. It is a required field and is typically specified in the VCF record.

* POS: The position of the variant on the chromosome. It is also a required field and is specified in the VCF record.

* ID: An optional identifier for the variant. If not provided, it can be left blank or set to a dot (".").

* REF: The reference allele(s) at the specified position. It is a required field.

* ALT: The alternate allele(s) at the specified position. It is a required field.

* QUAL: The Phred-scaled quality score. It represents the confidence in the variant call and is typically specified even if the value is missing or set to a default value.

* FILTER: The filter status for the variant, indicating whether it passed certain quality control filters. It is typically set to either "PASS" or a specific filter name if applied.

Below code is responsible for reading the meta data and available "custom" fields in the file to know what we can work with.

In [3]:
import vcf

with open(vcf_file, 'rb') as vcf_file_binary:
    vcf_reader = vcf.Reader(vcf_file_binary)

    for meta_info in vcf_reader.metadata:
        print("Metadata:", meta_info)

    column_headers = vcf_reader.infos.keys()
    print("Field Names:", column_headers)

Metadata: fileformat
Metadata: reference
Metadata: SentieonCommandLine.TNfilter
Metadata: SentieonCommandLine.TNhaplotyper2
Metadata: tumor_sample
Metadata: VEP
Metadata: EVE_CLASS
Metadata: EVE_SCORE
Metadata: CADD_PHRED
Metadata: CADD_RAW
Metadata: SpliceAI_pred_DP_AG
Metadata: SpliceAI_pred_DP_AL
Metadata: SpliceAI_pred_DP_DG
Metadata: SpliceAI_pred_DP_DL
Metadata: SpliceAI_pred_DS_AG
Metadata: SpliceAI_pred_DS_AL
Metadata: SpliceAI_pred_DS_DG
Metadata: SpliceAI_pred_DS_DL
Metadata: SpliceAI_pred_SYMBOL
Metadata: LOEUF
Metadata: PHENOTYPES
Metadata: NMD
Metadata: VEP-command-line
Field Names: odict_keys(['ACMG_class', 'ACMG_coding_impact', 'ACMG_gene', 'ACMG_rules', 'ACMG_score', 'ACMG_transcript', 'AMP_matches', 'AMP_rules', 'AMP_score', 'AMP_tier', 'AMP_total_samples', 'AS_FilterStatus', 'AS_SB_TABLE', 'CGDinheritance', 'ClinVarClass', 'ClinVarDisease', 'DANN_score', 'DP', 'ECNT', 'GERMQ', 'Gene', 'MBQ', 'MFRL', 'MMQ', 'MPOS', 'MutationTaster_pred', 'MutationTaster_score', 'POPAF'

Lastly, let's put it all together and read both "built-in" and "custom" fields.

In [4]:
i=0
with open(vcf_file, 'rb') as vcf_file_binary:
    vcf_reader = vcf.Reader(vcf_file_binary)

    for meta_info in vcf_reader.metadata:
        print("Metadata:", meta_info)

    column_headers = vcf_reader.infos.keys()
    print("Field Names:", column_headers)

    for record in vcf_reader:
        i+=1
        if i>10:
            break
        print("CHROM:", record.CHROM)
        print("POS:", record.POS)
        print("ID:", record.ID)
        print("REF:", record.REF)
        print("ALT:", record.ALT)
        print("QUAL:", record.QUAL)
        print("FILTER:", record.FILTER)

        for field in column_headers:
            info_value = record.INFO.get(field)
            if info_value is not None:
                print(f"{field}:", info_value)

        for sample in record.samples:
            print(f"Sample {sample.sample}: {sample['GT']}")

        print("\n")

Metadata: fileformat
Metadata: reference
Metadata: SentieonCommandLine.TNfilter
Metadata: SentieonCommandLine.TNhaplotyper2
Metadata: tumor_sample
Metadata: VEP
Metadata: EVE_CLASS
Metadata: EVE_SCORE
Metadata: CADD_PHRED
Metadata: CADD_RAW
Metadata: SpliceAI_pred_DP_AG
Metadata: SpliceAI_pred_DP_AL
Metadata: SpliceAI_pred_DP_DG
Metadata: SpliceAI_pred_DP_DL
Metadata: SpliceAI_pred_DS_AG
Metadata: SpliceAI_pred_DS_AL
Metadata: SpliceAI_pred_DS_DG
Metadata: SpliceAI_pred_DS_DL
Metadata: SpliceAI_pred_SYMBOL
Metadata: LOEUF
Metadata: PHENOTYPES
Metadata: NMD
Metadata: VEP-command-line
Field Names: odict_keys(['ACMG_class', 'ACMG_coding_impact', 'ACMG_gene', 'ACMG_rules', 'ACMG_score', 'ACMG_transcript', 'AMP_matches', 'AMP_rules', 'AMP_score', 'AMP_tier', 'AMP_total_samples', 'AS_FilterStatus', 'AS_SB_TABLE', 'CGDinheritance', 'ClinVarClass', 'ClinVarDisease', 'DANN_score', 'DP', 'ECNT', 'GERMQ', 'Gene', 'MBQ', 'MFRL', 'MMQ', 'MPOS', 'MutationTaster_pred', 'MutationTaster_score', 'POPAF'

#### Reading CSQ description 

In [31]:
import vcf

with open("data/EE_sample/EE_sample.vcf", 'r') as vcf_file:
    vcf_reader = vcf.Reader(vcf_file)

    # for key, val in vcf_reader.infos.items():
    #     print(f"KEY: {key:30} DESCR: {val}")

    print(vcf_reader.infos["CSQ"].desc[50:].split("|"))
    print(list(vcf_reader.infos.keys()))

['Allele', 'Consequence', 'IMPACT', 'SYMBOL', 'Gene', 'Feature_type', 'Feature', 'BIOTYPE', 'EXON', 'INTRON', 'HGVSc', 'HGVSp', 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation', 'DISTANCE', 'STRAND', 'FLAGS', 'SYMBOL_SOURCE', 'HGNC_ID', 'CANONICAL', 'MANE_SELECT', 'MANE_PLUS_CLINICAL', 'TSL', 'APPRIS', 'CCDS', 'ENSP', 'SWISSPROT', 'TREMBL', 'UNIPARC', 'UNIPROT_ISOFORM', 'REFSEQ_MATCH', 'SOURCE', 'REFSEQ_OFFSET', 'GIVEN_REF', 'USED_REF', 'BAM_EDIT', 'SIFT', 'PolyPhen', 'DOMAINS', 'HGVS_OFFSET', 'AF', 'AFR_AF', 'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF', 'gnomADe_AF', 'gnomADe_AFR_AF', 'gnomADe_AMR_AF', 'gnomADe_ASJ_AF', 'gnomADe_EAS_AF', 'gnomADe_FIN_AF', 'gnomADe_NFE_AF', 'gnomADe_OTH_AF', 'gnomADe_SAS_AF', 'gnomADg_AF', 'gnomADg_AFR_AF', 'gnomADg_AMI_AF', 'gnomADg_AMR_AF', 'gnomADg_ASJ_AF', 'gnomADg_EAS_AF', 'gnomADg_FIN_AF', 'gnomADg_MID_AF', 'gnomADg_NFE_AF', 'gnomADg_OTH_AF', 'gnomADg_SAS_AF', 'CLIN_SIG', 'SOMATIC', 'PHENO', 'PUBMED', '

In [35]:
import allel
data = allel.read_vcf("data/EE_sample/EE_sample.vcf")
data

{'samples': array(['200852'], dtype=object),
 'calldata/GT': array([[[0, 1]],
 
        [[0, 1]],
 
        [[0, 1]],
 
        [[0, 1]],
 
        [[0, 1]],
 
        [[0, 1]]], dtype=int8),
 'variants/ALT': array([['T', '', ''],
        ['A', '', ''],
        ['C', '', ''],
        ['A', '', ''],
        ['A', '', ''],
        ['T', '', '']], dtype=object),
 'variants/CHROM': array(['chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1'], dtype=object),
 'variants/FILTER_PASS': array([ True,  True,  True,  True,  True,  True]),
 'variants/ID': array(['rs2691315', 'rs201535981', 'rs71260069', 'rs367730352',
        'rs71267774', 'rs1396210256'], dtype=object),
 'variants/POS': array([ 15820,  17385,  17697, 133129, 183629, 184267]),
 'variants/QUAL': array([nan, nan, nan, nan, nan, nan], dtype=float32),
 'variants/REF': array(['G', 'G', 'G', 'G', 'G', 'C'], dtype=object)}

In [41]:
df = allel.vcf_to_dataframe("data/EE_sample/EE_sample.vcf", ["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"])
df

,CHROM,POS,ID,REF,ALT_1,ALT_2,ALT_3,QUAL,FILTER_PASS,FILTER_SB,...,gnomadGenomes_AC_Hemi_3,gnomadGenomes_AC_Hom_1,gnomadGenomes_AC_Hom_2,gnomadGenomes_AC_Hom_3,gnomadGenomes_AF_1,gnomadGenomes_AF_2,gnomadGenomes_AF_3,gnomadGenomes_AF_ethnic,hgvs,CSQ
0,chr1,15820,rs2691315,G,T,NaN,NaN,NaN,True,False,...,-1,18059,-1,-1,0.219648,NaN,NaN,0.093807,NaN,T|downstream_gene_variant|MODIFIER|DDX11L1|ENS...
1,chr1,17385,rs201535981,G,A,NaN,NaN,NaN,True,False,...,-1,21384,-1,-1,0.191644,NaN,NaN,0.213724,NaN,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...
2,chr1,17697,rs71260069,G,C,NaN,NaN,NaN,True,False,...,-1,17539,-1,-1,0.151133,NaN,NaN,0.178196,NaN,C|downstream_gene_variant|MODIFIER|DDX11L1|ENS...
3,chr1,133129,rs367730352,G,A,NaN,NaN,NaN,True,False,...,-1,72154,-1,-1,0.547875,NaN,NaN,0.597389,NaN,A|non_coding_transcript_exon_variant|MODIFIER|...
4,chr1,183629,rs71267774,G,A,NaN,NaN,NaN,True,False,...,-1,14902,-1,-1,0.133296,NaN,NaN,0.220891,NaN,A|downstream_gene_variant|MODIFIER|MIR6859-2|E...
5,chr1,184267,rs1396210256,C,T,NaN,NaN,NaN,True,False,...,-1,887,-1,-1,0.006127,NaN,NaN,0.006310,NaN,T|downstream_gene_variant|MODIFIER|MIR6859-2|E...


In [40]:
allel.vcf_to_csv("data/EE_sample/EE_sample.vcf", "data/EE_sample/EE_sample_allel.csv")